## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Get the data for Toronto</a>

2. <a href="#item2">Analyze Each Neighborhood</a>

3. <a href="#item3">Cluster Neighborhoods</a>

4. <a href="#item4">Examine Clusters</a>    
</font>
</div>

## 1 Get the data for Toronto
<h4>Import Pandas, Beautiful Soup and Requests libraries</h4> 

In [46]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from IPython.display import HTML, display
print('Libraries imported.')

Libraries imported.


<h4>Get the requred data and add it to dataframe</h4>
Get the page html from https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M. Parse it using Beautiful Soup, finding the table with the required data and create the dataframe

In [47]:
page_html = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(page_html, 'html.parser')
code_table = soup.find('table', {'class':'wikitable sortable'})
code_table_rows = code_table.find_all('tr')
rows = []
for tr in code_table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        rows.append(row)
df_loc = pd.DataFrame(rows, columns=['PostalCode', 'Borough', 'Neighborhood'])

<h4>Clean the dataframe</h4>
Add column names then remove all rows where we have <b>Not assigned</b> values in <b>Borough</b> colunm. Replace the <b>Not assigned</b> values in <b>Neighborhood</b> with the corresponding values of <b>Borough</b>.

In [48]:
df_loc = pd.DataFrame(rows, columns=['PostalCode', 'Borough', 'Neighborhood'])
df_loc = df_loc[df_loc.Borough != 'Not assigned']
#cols = df_loc.columns
#df_loc[cols] = df_loc[cols].mask(df_loc[cols].apply(lambda x: x.str.contains('Not assigned')), df_loc['Borough'], axis=0)

df_loc[['Neighborhood']] = df_loc[['Neighborhood']].mask(df_loc[['Neighborhood']].apply(lambda x: x.str.contains('Not assigned')), df_loc['Borough'], axis=0)

df_loc.index = range(len(df_loc))
df_loc.head(12)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [49]:
df_loc.shape

(212, 3)

<h4>Import Geospatial_data file</h4>
Read Geospatial_data file contaning the coordinates by postal code

In [50]:
filename = "http://cocl.us/Geospatial_data"
df_crd = pd.read_csv(filename, index_col=0)
df_crd = df_crd.reset_index()
df_crd.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


<h4>Add coordinates to Neighbourhood dataframe</h4>
Find respective coordinates by the Portal code in Geospatial_data dataframe and add it to Neighbourhood dataframe

In [51]:
for i in df_loc.index: 
    coordinates = df_crd.loc[df_crd['Postal Code'] ==  df_loc.iloc[i]['PostalCode'], ('Latitude', 'Longitude')]
    df_loc.loc[[0,i], 'Latitude'] = coordinates.Latitude.iloc[0]
    df_loc.loc[[0,i], 'Longitude'] = coordinates.Longitude.iloc[0]
df_loc.head(12)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.628841,-79.520999
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
4,M6A,North York,Lawrence Heights,43.718518,-79.464763
5,M6A,North York,Lawrence Manor,43.718518,-79.464763
6,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
7,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
8,M1B,Scarborough,Rouge,43.806686,-79.194353
9,M1B,Scarborough,Malvern,43.806686,-79.194353


## 2. Explore Neighborhoods in Toronto

In [52]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_loc['Borough'].unique()),
        df_loc.shape[0]
    )
)

The dataframe has 11 boroughs and 212 neighborhoods.


In [53]:
df_tor = df_loc[df_loc.Borough.str.contains('Toronto')].reset_index(drop=True)
df_tor.head(12)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
1,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
2,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937
3,M5B,Downtown Toronto,Garden District,43.657162,-79.378937
4,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
5,M4E,East Toronto,The Beaches,43.676357,-79.293031
6,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
7,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
8,M6G,Downtown Toronto,Christie,43.669542,-79.422564
9,M5H,Downtown Toronto,Adelaide,43.650571,-79.384568


In [54]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [73]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df_tor['Latitude'], df_tor['Longitude'], df_tor['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
#map_toronto
style_statement = '<style>.leaflet-control{color:#00FF00}</style>'
map_toronto.get_root().html.add_child(folium.Element(style_statement))
map_toronto

In [56]:
CLIENT_ID = 'BXHYB5NWY5RQOK3CUQR22OAFEJD0KTTBFNSFNG1MBE4MHXOX' # your Foursquare ID
CLIENT_SECRET = '1YQLIBQHCY345W1CDMRE2OBVJYSFEDUIHZ1NPZJ3RHO3P52O' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: BXHYB5NWY5RQOK3CUQR22OAFEJD0KTTBFNSFNG1MBE4MHXOX
CLIENT_SECRET:1YQLIBQHCY345W1CDMRE2OBVJYSFEDUIHZ1NPZJ3RHO3P52O


<h5>Create getNearbyVenues function to get nearby venues for all Toronto neighborhoods</h5>

In [57]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    LIMIT = 100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

<h5>Call getNearbyVenues function on each neighborhood and create a new dataframe called toronto_venues</h5>

In [58]:
toronto_venues = getNearbyVenues(names=df_tor['Neighborhood'],
                                   latitudes=df_tor['Latitude'],
                                   longitudes=df_tor['Longitude']
                                  )

Harbourfront
Regent Park
Ryerson
Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide
King
Richmond
Dovercourt Village
Dufferin
Harbourfront East
Toronto Islands
Union Station
Little Portugal
Trinity
The Danforth West
Riverdale
Design Exchange
Toronto Dominion Centre
Brockton
Exhibition Place
Parkdale Village
The Beaches West
India Bazaar
Commerce Court
Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North
Forest Hill West
High Park
The Junction South
North Toronto West
The Annex
North Midtown
Yorkville
Parkdale
Roncesvalles
Davisville
Harbord
University of Toronto
Runnymede
Swansea
Moore Park
Summerhill East
Chinatown
Grange Park
Kensington Market
Deer Park
Forest Hill SE
Rathnelly
South Hill
Summerhill West
CN Tower
Bathurst Quay
Island airport
Harbourfront West
King and Spadina
Railway Lands
South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown
St. James Town
First Canadian Place
Underground city


<h5>Check the size of the resulting dataframe, how many venues were returned for each neighborhood and how many unique categories can be curated from all the returned venues</h5>

In [59]:
print(toronto_venues.shape)
toronto_venues.head()

(3271, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Harbourfront,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Harbourfront,43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,Harbourfront,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,Harbourfront,43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


In [60]:
toronto_venues.groupby('Neighborhood').count()
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 236 uniques categories.


## 3. Analyze Each Neighborhood

In [61]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<h4>Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category and print each neighborhood along with the top 5 most common venues</h4>

In [62]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
num_top_venues = 5
for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide----
             venue  freq
0      Coffee Shop  0.06
1             Café  0.05
2       Steakhouse  0.04
3  Thai Restaurant  0.04
4              Bar  0.04


----Bathurst Quay----
              venue  freq
0   Airport Service  0.14
1  Airport Terminal  0.14
2    Airport Lounge  0.14
3          Boutique  0.07
4     Boat or Ferry  0.07


----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.05
2  Italian Restaurant  0.03
3          Restaurant  0.03
4      Farmers Market  0.03


----Brockton----
                   venue  freq
0         Breakfast Spot  0.09
1  Performing Arts Venue  0.09
2            Coffee Shop  0.09
3                   Café  0.09
4      Convenience Store  0.04


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0  Light Rail Station  0.12
1         Yoga Studio  0.06
2       Auto Workshop  0.06
3       Garden Center  0.06
4              Garden  0.06


----CN Tower----
           

<h4>Create a function to sort the venues in descending order and put, create the new dataframe and display the top 10 venues for each neighborhood.</h4>

In [63]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [64]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Café,Steakhouse,Bar,Thai Restaurant,Burger Joint,American Restaurant,Gym,Bakery,Hotel
1,Bathurst Quay,Airport Lounge,Airport Terminal,Airport Service,Boat or Ferry,Sculpture Garden,Harbor / Marina,Plane,Boutique,Airport Gate,Airport
2,Berczy Park,Coffee Shop,Cocktail Bar,Italian Restaurant,Café,Seafood Restaurant,Cheese Shop,Farmers Market,Restaurant,Steakhouse,Bakery
3,Brockton,Café,Performing Arts Venue,Breakfast Spot,Coffee Shop,Bar,Restaurant,Stadium,Caribbean Restaurant,Italian Restaurant,Furniture / Home Store
4,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Garden,Skate Park,Park,Farmers Market,Fast Food Restaurant,Brewery,Burrito Place,Restaurant


## 3. Cluster Neighborhoods

<h4>Run *k*-means to cluster the neighborhood into 5 clusters.</h>

In [65]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 2, 0, 0, 0, 2, 0, 0, 0, 0], dtype=int32)

<h4>Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.</h4>

In [66]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_tor

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,0,Coffee Shop,Café,Pub,Park,Bakery,Mexican Restaurant,Breakfast Spot,Theater,Event Space,Asian Restaurant
1,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636,0,Coffee Shop,Café,Pub,Park,Bakery,Mexican Restaurant,Breakfast Spot,Theater,Event Space,Asian Restaurant
2,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937,0,Clothing Store,Coffee Shop,Café,Cosmetics Shop,Middle Eastern Restaurant,Restaurant,Theater,Italian Restaurant,Tea Room,Bubble Tea Shop
3,M5B,Downtown Toronto,Garden District,43.657162,-79.378937,0,Clothing Store,Coffee Shop,Café,Cosmetics Shop,Middle Eastern Restaurant,Restaurant,Theater,Italian Restaurant,Tea Room,Bubble Tea Shop
4,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Restaurant,Café,Bakery,Breakfast Spot,Italian Restaurant,Park,Hotel,Clothing Store,Gastropub


<h4>Show resulting clusters</h4>

In [67]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 4. Examine Clusters

<h4>Cluster 1</h4>

In [68]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Café,Pub,Park,Bakery,Mexican Restaurant,Breakfast Spot,Theater,Event Space,Asian Restaurant
1,Downtown Toronto,0,Coffee Shop,Café,Pub,Park,Bakery,Mexican Restaurant,Breakfast Spot,Theater,Event Space,Asian Restaurant
2,Downtown Toronto,0,Clothing Store,Coffee Shop,Café,Cosmetics Shop,Middle Eastern Restaurant,Restaurant,Theater,Italian Restaurant,Tea Room,Bubble Tea Shop
3,Downtown Toronto,0,Clothing Store,Coffee Shop,Café,Cosmetics Shop,Middle Eastern Restaurant,Restaurant,Theater,Italian Restaurant,Tea Room,Bubble Tea Shop
4,Downtown Toronto,0,Coffee Shop,Restaurant,Café,Bakery,Breakfast Spot,Italian Restaurant,Park,Hotel,Clothing Store,Gastropub
5,East Toronto,0,Health Food Store,Coffee Shop,Pub,Dim Sum Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
6,Downtown Toronto,0,Coffee Shop,Cocktail Bar,Italian Restaurant,Café,Seafood Restaurant,Cheese Shop,Farmers Market,Restaurant,Steakhouse,Bakery
7,Downtown Toronto,0,Coffee Shop,Café,Italian Restaurant,Bar,Burger Joint,Sushi Restaurant,Bubble Tea Shop,Ice Cream Shop,Thai Restaurant,Spa
8,Downtown Toronto,0,Grocery Store,Café,Park,Restaurant,Baby Store,Italian Restaurant,Athletics & Sports,Nightclub,Diner,Coffee Shop
9,Downtown Toronto,0,Coffee Shop,Café,Steakhouse,Bar,Thai Restaurant,Burger Joint,American Restaurant,Gym,Bakery,Hotel


<h4>Cluster 2</h4>

In [69]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,Central Toronto,1,Garden,Women's Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


<h4>Cluster 3</h4>

In [70]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
59,Downtown Toronto,2,Airport Lounge,Airport Terminal,Airport Service,Boat or Ferry,Sculpture Garden,Harbor / Marina,Plane,Boutique,Airport Gate,Airport
60,Downtown Toronto,2,Airport Lounge,Airport Terminal,Airport Service,Boat or Ferry,Sculpture Garden,Harbor / Marina,Plane,Boutique,Airport Gate,Airport
61,Downtown Toronto,2,Airport Lounge,Airport Terminal,Airport Service,Boat or Ferry,Sculpture Garden,Harbor / Marina,Plane,Boutique,Airport Gate,Airport
62,Downtown Toronto,2,Airport Lounge,Airport Terminal,Airport Service,Boat or Ferry,Sculpture Garden,Harbor / Marina,Plane,Boutique,Airport Gate,Airport
63,Downtown Toronto,2,Airport Lounge,Airport Terminal,Airport Service,Boat or Ferry,Sculpture Garden,Harbor / Marina,Plane,Boutique,Airport Gate,Airport
64,Downtown Toronto,2,Airport Lounge,Airport Terminal,Airport Service,Boat or Ferry,Sculpture Garden,Harbor / Marina,Plane,Boutique,Airport Gate,Airport
65,Downtown Toronto,2,Airport Lounge,Airport Terminal,Airport Service,Boat or Ferry,Sculpture Garden,Harbor / Marina,Plane,Boutique,Airport Gate,Airport


<h4>Cluster 4</h4>

In [71]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
49,Central Toronto,3,Playground,Gym,Tennis Court,Dessert Shop,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
50,Central Toronto,3,Playground,Gym,Tennis Court,Dessert Shop,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


<h4>Cluster 5</h4>

In [72]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
34,Central Toronto,4,Mexican Restaurant,Trail,Jewelry Store,Sushi Restaurant,Women's Store,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
35,Central Toronto,4,Mexican Restaurant,Trail,Jewelry Store,Sushi Restaurant,Women's Store,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
